概要

1.  Gitリポジトリをクローンし、コードディレクトリへ移動。
2.  Ollama環境（テスト対象LLM）をセットアップし、モデルをプル。
3.  必要なPythonライブラリをインストール。
4.  Base LLM (RAGなし) の回答生成スクリプトを実行。
5.  RAGありの回答生成スクリプトを実行。
6.  **評価スクリプトを実行。**
7.  生成された回答と評価結果を確認。

---

# -*- coding: utf-8 -*-
"""
宿題実行用 Colab ノートブック (Base LLM & RAG & 評価) - Ollama + deepseek-r1:1.5b / Gemini Flash Evaluator

このノートブックでは、Colab VM上に環境をセットアップし、
ローカルのPythonスクリプトを実行して
Base LLM (RAGなし) と RAG あり の回答を生成し、さらに Gemini Flash を評価者としてその回答を評価します。
テスト対象モデルは Ollama の deepseek-r1:1.5b です。
"""

In [1]:
# @title 1. Git リポジトリのクローンと移動

# 以前にディレクトリを変更している可能性があるので、まずは /content/ に戻る
%cd /content/

# 既に存在するリポジトリディレクトリを削除
# -r: ディレクトリを再帰的に削除
# -f: 確認なしで強制的に削除
#!rm -rf /content/LLM_RAG_Homework
#print("/content/LLM_RAG_Homework ディレクトリを削除しました。")

github_username = "ikdmail"
repository_name = "LLM_RAG_Homework"

print(f"GitHubリポジトリ '{github_username}/{repository_name}' をクローンします。")
!git clone https://github.com/{github_username}/{repository_name}.git

print("\nクローンが完了しました。")

# クローンしたリポジトリ内の 'code' ディレクトリに移動します。
# base_llm_answer_generator.py, rag_answer_generator.py, evaluate_answers.py スクリプトが
# 相対パス (../questions/questions.txt など) を参照するため、
# これらのスクリプトがあるディレクトリに移動して実行する必要があります。
import os

repository_name = "LLM_RAG_Homework" # 上記と同じリポジトリ名
code_dir = os.path.join("/content", repository_name, "code")

# カレントディレクトリを変更
%cd {code_dir}

print(f"カレントディレクトリを '{os.getcwd()}' に変更しました。")
# 現在のディレクトリの内容を確認（任意）
# !ls -al
# !ls -al ../questions/ # 質問ファイルがあるか確認
# !ls -al ../references/ # 参照資料があるか確認

/content
GitHubリポジトリ 'ikdmail/LLM_RAG_Homework' をクローンします。
Cloning into 'LLM_RAG_Homework'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 47 (delta 19), reused 38 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 36.27 KiB | 651.00 KiB/s, done.
Resolving deltas: 100% (19/19), done.

クローンが完了しました。
/content/LLM_RAG_Homework/code
カレントディレクトリを '/content/LLM_RAG_Homework/code' に変更しました。


```markdown
# @title 2. Ollama 環境のセットアップ (テスト対象LLM)

Colab VM上にOllamaサーバーをセットアップし、テスト対象のLLMモデル (`deepseek-r1:1.5b`) をダウンロード（プル）します。
**注意:** Colabのランタイムは一時的であり、セッションが終了するとVMの状態はリセットされます。
ランタイムが中断または再起動された場合は、これらのセルを再度実行する必要があります。

以下のセルを順番に実行してください。
```

In [2]:
# @title 2.1. Ollama のインストール

# Colab VM (Linux環境) に Ollama をダウンロードしインストールします。
print("Ollamaをインストールします...")
!curl -fsSL https://ollama.com/install.sh | sh

print("\nOllamaのインストールが完了しました。")
# インストールされたか確認（任意）
# !ollama --version

Ollamaをインストールします...
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

Ollamaのインストールが完了しました。


In [3]:
# @title 2.2. Ollama サーバーの起動

# Ollama サーバーをバックグラウンドで起動します。
# "&" を末尾につけることで、サーバーを起動しつつ次のコマンドに進むことができます。
print("Ollamaサーバーをバックグラウンドで起動します...")
!nohup ollama serve &

# サーバーが起動するまで少し待つ（推奨）
# サーバーが完全に起動する前に次のプルコマンドを実行するとエラーになることがあります。
import time
print("サーバー起動のため15秒待機します...") # モデルが小さいので15秒で十分なはず
time.sleep(15)
print("待機完了。")

# サーバーが起動したか簡易的に確認するコマンド（任意）
# !ollama list # モデルリストが表示されればサーバーは起動しています

Ollamaサーバーをバックグラウンドで起動します...
nohup: appending output to 'nohup.out'
サーバー起動のため15秒待機します...
待機完了。


In [4]:
# @title 2.3. 使用する Ollama モデルのプル (ダウンロード)

# TODO: ここに使用したいOllamaモデル名を指定してください。
# このモデル名は、スクリプト実行時に引数として渡します。
ollama_model_name = "deepseek-r1:1.5b" # 例: "deepseek-r1:1.5b", "mistral", "gemma" など

print(f"Ollamaモデル '{ollama_model_name}' をプルします...")
!ollama pull {ollama_model_name}

print(f"\nモデル '{ollama_model_name}' のプルが完了しました。")

# プルしたモデルを確認（任意）
# !ollama list

Ollamaモデル 'deepseek-r1:1.5b' をプルします...
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling aabd4debf0c8:   1% ▕▏  11 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   8% ▕▏  88 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  11% ▕▏ 126 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  15% ▕▏ 170 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  19% ▕▏ 217 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  21% ▕▏ 236 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  26% ▕▏ 294 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  32% ▕▏ 354 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  36% ▕▏ 398 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  36% ▕▏ 403 MB/1.1 GB                  

In [5]:
# @title 3. スクリプトが必要とする Python ライブラリのインストール

# base_llm_answer_generator.py, rag_answer_generator.py, evaluate_answers.py
# が使用するライブラリをインストールします。
# ollama, sentence-transformers, numpy, scikit-learn, pandas, google-generativeai が必要です。
print("必要なPythonライブラリをインストールします...")
!pip install ollama sentence-transformers numpy scikit-learn pandas google-generativeai

print("\n必要なPythonライブラリのインストールが完了しました。")

必要なPythonライブラリをインストールします...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successful

```markdown
# @title 4. Base LLM (RAGなし) スクリプトの実行

Base LLM (Ollama deepseek-r1:1.5b) を使用して、RAGなしで回答を生成するスクリプトを実行します。
スクリプトは `base_llm_answer_generator.py` です。
```

In [6]:
# @title 4.1. base_llm_answer_generator.py スクリプトの実行

# base_llm_answer_generator.py スクリプトを実行して回答を生成します。
# --model_name 引数で、使用するOllamaモデル名を渡します。
# スクリプトはカレントディレクトリ (code/) から相対パスで
# ../questions/questions.txt を読み込み、../results/raw_answers_base_llm.md に保存します。

# TODO: セル 2.3 でプルしたモデル名を指定してください。
ollama_model_to_use = ollama_model_name # 例: "deepseek-r1:1.5b"

print(f"base_llm_answer_generator.py スクリプトを実行します (モデル: {ollama_model_to_use})...")
# 現在のカレントディレクトリ (../LLM_RAG_Homework/code) からスクリプトを実行し、引数を渡す
!python base_llm_answer_generator.py --model_name {ollama_model_to_use}

print("\nベースLLM (RAGなし) による回答生成が完了しました。")

base_llm_answer_generator.py スクリプトを実行します (モデル: deepseek-r1:1.5b)...
Colab + Ollama 環境でBase LLM回答生成スクリプトを実行中...
カレントディレクトリ: /content/LLM_RAG_Homework/code
使用モデル（引数より）: deepseek-r1:1.5b
LLMクライアントを初期化します...
Ollama client initialized successfully for model 'deepseek-r1:1.5b' at http://localhost:11434.
'../questions/questions.txt' から質問を読み込んでいます...
10 件の質問を読み込みました。
Ollama (deepseek-r1:1.5b) を使用して回答を生成します...
(1/10) 質問処理中...
--- 質問 ---
宇宙イノベーションズ株式会社の星屑配達プロジェクトにおける、2025年第2四半期の最優先目標は何ですか？
--- ベースLLM回答 (deepseek-r1:1.5b / 先頭のみ表示) ---
<think>  </think>  对不起，我还没有学会回答这个问题。如果你有其他问题，我非常乐意为你提供帮助。
------------------------------
(2/10) 質問処理中...
--- 質問 ---
プロジェクトで使用される量子ふるい技術の主任開発者は誰ですか？
--- ベースLLM回答 (deepseek-r1:1.5b / 先頭のみ表示) ---
<think> 嗯，我现在要回答关于在项目中使用的量子计算机和量子人工智能的主要开发者的问题。我需要先理解什么是量子计算机和人工智能，以及它们如何在现代应用中被利用。  首先，量子计算是什么呢？我记得它是利用量子力学中的基本原理来处理信息的技术。与经典计算机不同，它可以在相同的硬件下运行更多的运算，...
------------------------------
(3/10) 質問処理中...
--- 質問 ---
個人向け星屑パッケージ「プチ・コモス」の、最小内容量は何立方センチメートルですか？
--- ベースLLM回答 (deepseek-

```markdown
# @title 5. RAG あり スクリプトの実行

RAG (Retrieval Augmented Generation) を使用して回答を生成するスクリプトを実行します。
スクリプトは `rag_answer_generator.py` です。

```

In [7]:
# @title 5.1. rag_answer_generator.py スクリプトの実行

# rag_answer_generator.py スクリプトを実行してRAG回答を生成します。
# --model_name 引数で、使用するOllamaモデル名を渡します。
# スクリプトはカレントディレクトリ (code/) から相対パスで
# ../questions/questions.txt および ../references/ の資料を読み込み、
# ../results/raw_answers_rag.md に保存します。

# TODO: セル 2.3 でプルしたモデル名を指定してください。
ollama_model_to_use = ollama_model_name # 例: "deepseek-r1:1.5b"

print(f"rag_answer_generator.py スクリプトを実行します (モデル: {ollama_model_to_use})...")
# 現在のカレントディレクトリ (../LLM_RAG_Homework/code) からスクリプトを実行し、引数を渡す
!python rag_answer_generator.py --model_name {ollama_model_to_use}

print("\nRAG あり による回答生成が完了しました。")

rag_answer_generator.py スクリプトを実行します (モデル: deepseek-r1:1.5b)...
2025-05-06 02:01:48.314619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746496908.557831    1384 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746496908.624187    1384 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 02:01:49.168493: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Colab + Ollama 環境でRAG回答生成スクリプトを実行中...
カレントディレクトリ: /con

```markdown
# @title 6. 回答の評価 (Gemini Flash 評価者)

生成された Base LLM 回答と RAG 回答を、Gemini Flash モデルを評価者として使用し評価します。
スクリプトは `evaluate_answers.py` です。

**事前準備:**
1. Gemini API キーを準備します。
2. Colabの左サイドバーにある鍵アイコンの「シークレット」機能を開きます。
3. 「新しいシークレット」を作成し、名前を `EVALUATOR_GOOGLE_API_KEY` とし、値にあなたのGemini APIキーを貼り付けます。
4. 「ノートブックでのアクセスを許可」をオンにします。
5. **`../questions/correct_answers.json` ファイルに、全ての質問に対する正確な回答がJSON形式で記述されていること**を確認してください。
6. `evaluate_answers.py` スクリプト内の `EVALUATOR_LLM_MODEL` が `gemini-1.5-flash-latest` に設定されていることを確認してください。
```

In [8]:
# @title 5.evaluate_answers.py スクリプトの内容表示 (任意)

# 作成した evaluate_answers.py スクリプトの内容をノートブック上に表示します。
# 実行前にコードを確認したい場合に便利です。
print("evaluate_answers.py の内容:")
# 現在のカレントディレクトリ (../LLM_RAG_Homework/code) からファイルを参照
!cat evaluate_answers.py

evaluate_answers.py の内容:
# evaluate_answers.py (Gemini Flash評価 & 平均スコア版 - 外部JSONから正解読み込み - リトライ対応)

import os
import re
import json
import pandas as pd
import time
import random # リトライ待機時間のばらつきのためにインポート

# --- Import necessary library for the Evaluator LLM ---
import google.generativeai as genai
from google.api_core import exceptions as google_exceptions # Google APIのエラーハンドリング用

# --- Configuration ---
REPO_ROOT = os.path.join("..")
QUESTIONS_FILE = os.path.join(REPO_ROOT, "questions", "questions.txt")
CORRECT_ANSWERS_FILE = os.path.join(REPO_ROOT, "questions", "correct_answers.json")
BASE_LLM_ANSWERS_FILE = os.path.join(REPO_ROOT, "results", "raw_answers_base_llm.md")
RAG_ANSWERS_FILE = os.path.join(REPO_ROOT, "results", "raw_answers_rag.md")
OUTPUT_DIR = os.path.join(REPO_ROOT, "results")
OUTPUT_EVALUATION_FILE = os.path.join(OUTPUT_DIR, "evaluation_results.csv")

# --- Automated Evaluation Settings ---
ENABLE_AUTOMATED_EVALUATION = True
EVALUATOR_API_KEY = os.getenv("EVALUATOR_GOOGL

In [13]:
# @title 6.1. evaluate_answers.py スクリプトの実行

# evaluate_answers.py スクリプトを実行し、生成された回答を評価します。
# スクリプトは ../results/ ディレクトリの回答ファイルを読み込み、
# 同じディレクトリに evaluation_results.csv を出力します。
# 環境変数 EVALUATOR_GOOGLE_API_KEY が設定されている必要があります。
from getpass import getpass
import os
from google.colab import userdata
os.environ["EVALUATOR_GOOGLE_API_KEY"] = userdata.get("EVALUATOR_GOOGLE_API_KEY")

print("evaluate_answers.py スクリプトを実行して回答を評価します...")
# 現在のカレントディレクトリ (../LLM_RAG_Homework/code) からスクリプトを実行
!python evaluate_answers.py

print("\n回答評価スクリプトの実行が完了しました。評価結果がCSVファイルに保存されました。")

evaluate_answers.py スクリプトを実行して回答を評価します...
Colab環境で回答評価スクリプトを実行中 (Gemini Flash 評価者)...
カレントディレクトリ: /content/LLM_RAG_Homework/code
'../questions/correct_answers.json' から正解データを読み込んでいます...
Correct answers loaded successfully from ../questions/correct_answers.json
10 件の正解データを読み込みました。

評価者LLM (gemini-1.5-flash-latest) を初期化します...
評価者LLMクライアント (gemini-1.5-flash-latest) の初期化に成功しました。

生成された回答ファイルを読み込みます...
評価対象となる 10 件の質問を評価します。

回答の評価を実行します (自動評価: True)...

--- 質問 1/10 の評価 ---
質問: 宇宙イノベーションズ株式会社の星屑配達プロジェクトにおける、2025年第2四半期の最優先目標は何ですか？...
  -> Running automated evaluation using 2 prompts...
    -> Evaluating with Prompt Template 1...
      -> BaseLLM...
        -> Success on attempt 1.
      -> RAG...
        -> Success on attempt 1.
    -> Waiting 1.00 seconds before next prompt...
    -> Evaluating with Prompt Template 2...
      -> BaseLLM...
        -> Success on attempt 1.
      -> RAG...
        -> Success on attempt 1.

Waiting 7.00 seconds before next question...

--- 質問 2/10 の評価 ---
質問: プ

```markdown
# @title 7. 生成された回答と評価結果の確認

生成された回答ファイルおよび評価結果ファイルは、指定した出力ファイルに保存されています。
- Base LLM (RAGなし) 回答: `../results/raw_answers_base_llm.md`
- RAG あり 回答: `../results/raw_answers_rag.md`
- 評価結果 (CSV): `../results/evaluation_results.csv`

以下のセルで、それぞれのファイルの内容を表示してみましょう。
```

In [14]:
# @title 7.1. Base LLM (RAGなし) 回答ファイルの内容表示

output_file_base = "../results/raw_answers_base_llm.md"
print(f"'{output_file_base}' の内容:")

# ファイルが存在するか確認
if os.path.exists(output_file_base):
    # ファイルの内容を表示
    !cat {output_file_base}
else:
    print("エラー: Base LLM 回答ファイルが見つかりません。スクリプトが正常に実行されたか確認してください。")
    # 想定される保存先ディレクトリの内容を確認（任意）
    # !ls -al ../results/

'../results/raw_answers_base_llm.md' の内容:
# ベースLLM (Colab + Ollama - モデル: deepseek-r1:1.5b) による回答 (RAGなし)

使用Ollamaモデル: deepseek-r1:1.5b
Ollama Host: http://localhost:11434

このファイルには、RAG（検索拡張生成）を使用せず、指定されたOllamaモデルの知識のみに基づいて生成された回答が格納されています。

---

## 質問 1

**質問:** 宇宙イノベーションズ株式会社の星屑配達プロジェクトにおける、2025年第2四半期の最優先目標は何ですか？

**ベースLLM回答:**
<think>

</think>

对不起，我还没有学会回答这个问题。如果你有其他问题，我非常乐意为你提供帮助。

---

## 質問 2

**質問:** プロジェクトで使用される量子ふるい技術の主任開発者は誰ですか？

**ベースLLM回答:**
<think>
嗯，我现在要回答关于在项目中使用的量子计算机和量子人工智能的主要开发者的问题。我需要先理解什么是量子计算机和人工智能，以及它们如何在现代应用中被利用。

首先，量子计算是什么呢？我记得它是利用量子力学中的基本原理来处理信息的技术。与经典计算机不同，它可以在相同的硬件下运行更多的运算，这可能带来巨大的效率提升。所以，谁是主要的开发者呢？

我知道深度学习和机器学习等领域的发展离不开深度神经网络，而这些网络通常依赖于大量的计算资源。也许量子人工智能团队就是用来开发这些技术的？或者是不是还有其他的公司更知名？

然后是量子计算机本身。量子计算机需要特别的硬件来实现，比如qubit数量、高可靠性以及低能耗等特性。谁负责研发这些硬件呢？Ising机或者超导体芯片可能是个好例子。

另外，我还听说过一些公司比如IBM、Dequant、Cirq和Qiskit。他们是不是主要开发了相关领域的软件或硬件？

让我再想想，深度学习领域是否有专门的团队？我好像看到很多公司的深度学习框架，比如TensorFlow、PyTorch等，这些是用于机器学习的工具，而它们背后的支持可能涉及到量子计算吗？或者这是另一个不同的领域。

还有，一些公司的联合实验室可能在开发这些技术。

In [15]:
# @title 7.2. RAG あり 回答ファイルの内容表示

output_file_rag = "../results/raw_answers_rag.md"
print(f"'{output_file_rag}' の内容:")

# ファイルが存在するか確認
if os.path.exists(output_file_rag):
    # ファイルの内容を表示
    !cat {output_file_rag}
else:
    print("エラー: RAG 回答ファイルが見つかりません。スクリプトが正常に実行されたか確認してください。")
    # 想定される保存先ディレクトリの内容を確認（任意）
    # !ls -al ../results/

'../results/raw_answers_rag.md' の内容:
# RAGによる回答 (Colab + Ollama - モデル: deepseek-r1:1.5b)

使用LLM: Ollama (deepseek-r1:1.5b)
使用Embeddingモデル: all-MiniLM-L6-v2
RAGパラメータ: Chunk Size=500, Overlap=50, Retrieved Chunks=3

このファイルには、RAG（検索拡張生成）を使用して生成された回答が格納されています。

---

## 質問 1

**質問:** 宇宙イノベーションズ株式会社の星屑配達プロジェクトにおける、2025年第2四半期の最優先目標は何ですか？

**RAG回答:**
<think>
まず、質問は宇宙イノ bevianz株式会社の星屑配達プロジェクトについて、2025年第2四半期の最優先目標を求める question。
参考情報の開始部分から、プロジェクト全体の timing lineで2025第2四半期の最優先目標は、新型集塵船「オリオンの雫」が予定されている流星群-throughイベントに合わせて成功させることを説明されています。重点在于希望 achieves 18月14日予定istent。

参考情報の終了部分では、この信息を整合し、回答者に反映します。
</think>

2025年第2四半期の最優先目標は、新型集塵船「オリオンの雫」が予定されている流星群-throughイベントに合わせて成功させrmです。

---

## 質問 2

**質問:** プロジェクトで使用される量子ふるい技術の主任開発者は誰ですか？

**RAG回答:**
<think>
まず、質問が「oprojectで使用される量子ふるい技術の主任開発者は Whoですか？」です。
参考情報では、プロジェクトの最優先目標は2025年第2四半期の最優先目標を、新型集塵船「オリオンの雫」の最終軌道テストを成功させることです。
技術開発の側面では、核となる量子ふるい技術が注意されています。特に、**量子ふるい技術**について、**入社2年目の若手エンジニア、エリア・ヴァンス氏**が主任開発者としてチームを力強く牽引しています。
したがって、 quantum foreshortage t

In [16]:
# @title 7.3. 評価結果 (CSV) ファイルの内容表示

# CSVファイルはExcelなどで開いて確認するのが便利ですが、Colabでも表示できます。
# pandasを使って表示してみましょう。
import pandas as pd

output_file_eval = "../results/evaluation_results.csv"
print(f"'{output_file_eval}' の内容:")

# ファイルが存在するか確認
if os.path.exists(output_file_eval):
    try:
        df_results = pd.read_csv(output_file_eval, encoding='utf-8')
        display(df_results) # DataFrameを表示
    except Exception as e:
        print(f"エラー: CSVファイルの読み込みまたは表示に失敗しました: {e}")
else:
    print("エラー: 評価結果ファイルが見つかりません。スクリプトが正常に実行されたか確認してください。")
    # 想定される保存先ディレクトリの内容を確認（任意）
    # !ls -al ../results/

'../results/evaluation_results.csv' の内容:


,Question,Correct_Answer,BaseLLM_Answer,RAG_Answer,BaseLLM_Accuracy_Score_Avg,BaseLLM_Completeness_Score_Avg,BaseLLM_Relevance_Score_Avg,RAG_Accuracy_Score_Avg,RAG_Completeness_Score_Avg,RAG_Relevance_Score_Avg,BaseLLM_Evaluation_Notes,RAG_Evaluation_Notes
0,宇宙イノベーションズ株式会社の星屑配達プロジェクトにおける、2025年第2四半期の最優先目標...,星屑配達プロジェクトの2025年第2四半期の最優先目標は、新型集塵船「オリオンの雫」の最終軌...,<think>\n\n</think>\n\n对不起，我还没有学会回答这个问题。如果你有其他...,<think>\nまず、質問は宇宙イノ bevianz株式会社の星屑配達プロジェクトについて...,0.0,0.0,0.0,4.0,3.5,4.5,Avg over 2/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...
1,プロジェクトで使用される量子ふるい技術の主任開発者は誰ですか？,量子ふるい技術の主任開発者は、入社2年目の若手エンジニア、エリア・ヴァンス氏です。,<think>\n嗯，我现在要回答关于在项目中使用的量子计算机和量子人工智能的主要开发者的问...,<think>\nまず、質問が「oprojectで使用される量子ふるい技術の主任開発者は W...,0.0,0.0,0.5,3.0,3.5,3.5,Avg over 2/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...
2,個人向け星屑パッケージ「プチ・コモス」の、最小内容量は何立方センチメートルですか？,個人向け星屑パッケージ「プチ・コモス」の最小内容量は、安全基準に基づき3立方センチメートルと...,<think>\n嗯，我现在要找出一个个人用途的星屑(Puichi)品牌推出的包裝“Co M...,<think>\nまず、ユーザーが質問做的事情 것은「独自な星屑パッケージ「プチ・CoMoS...,0.5,0.0,1.0,3.5,3.5,4.5,Avg over 2/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...
3,2025年5月1日付の社内広報によると、星屑配達プロジェクトが最近達成した重要なマイルストー...,2025年5月1日付の社内広報によれば、星屑配達プロジェクトは、収集した星屑の銀河間輸送にお...,<think>\n好，我现在要回答用户的问题：“2025年5月1日付の社内広报によると、星屑...,<think>\nまず、ユーザーが问題を追加しているので firstlineで refere...,0.0,0.0,1.0,0.5,0.0,1.5,Avg over 1/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...
4,ベータグレードの星屑を処理する際の、必須安全手順プロトコルは何番ですか？,ベータグレードの星屑を処理する際の必須安全手順プロトコルは「プロトコル7B：低重力環境下にお...,<think>\n好，我现在要分析用户的问题。用户问的是处理Beta Greet的星形图标时...,<think>\nまず、ユーザーが问done做的事情 looks at the provid...,0.0,0.0,0.0,1.0,1.0,2.5,Avg over 2/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...
5,マーケティング部門が承認した、初期プロモーションキャンペーンの公式キャッチコピーは何ですか？,初期プロモーションキャンペーンの公式キャッチコピーは、「あなたの願いを、宇宙からひと摘み。」...,<think>\n嗯，用户问的是 Marketing Department 官方给出的最初的...,<think>\nまず、参考情報を見ると、宇宙イノベーションズ株式会社は2025年4月20日...,0.0,0.0,1.5,0.5,0.0,1.0,Avg over 2/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...
6,社内規定「ユニバース・コネクションズ」第5条第3項に基づくと、星屑配達プロジェクトのメンバー...,社内規定「ユニバース・コネクションズ」第5条第3項に基づくと、星屑配達プロジェクトのメンバー...,<think>\n\n</think>\n\n对不起，我还没有学会回答这个问题。如果你有其他...,<think>\nまず、質問者に问 designingは「星屑配達プロジェクトのメンバーが奥...,0.0,0.0,0.0,2.5,3.5,3.5,Avg over 2/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...
7,プロジェクトの次のフェーズで導入が検討されている、顧客とのインタラクションに利用する新しいテ...,プロジェクトの次のフェーズで導入が検討されている、顧客とのインタラクションに利用する新しいテ...,"<think>\nOkay, so the user is asking about a n...",<think>\n好的，我需要根据提供的参考信息来回答“プロジェクトの次のフェーズで導入が検...,0.0,0.0,1.0,0.0,0.0,0.0,Avg over 2/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...
8,宇宙イノベーションズ株式会社が倫理的な側面で協力している外部の専門機関は何ですか？,宇宙イノベーションズ株式会社が倫理的な側面、特に天体物質の商業化における影響について協力して...,<think>\n\n</think>\n\n对不起，我还没有学会回答这个问题。如果你有其他...,<think>\nまず、ユーザーが提出された質問は「宇宇宙イノベーションズ株式会社が倫理的な...,0.0,0.0,0.0,4.0,3.5,4.0,Avg over 2/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...
9,2025年4月20日に発生したアルファ収集テスト中の予期せぬ事象とは具体的に何でしたか？,2025年4月20日に発生したアルファ収集テスト中の予期せぬ事象は、収集ポッドの近くで一時的...,<think>\n\n</think>\n\n对不起，我还没有学会回答这个问题。如果你有其他...,<think>\nまず、参考情報を reference看一下します。参考情報は2025年4月...,0.0,0.0,0.0,4.0,4.0,4.5,Avg over 2/6 total attempts. Reasons: P1 Attem...,Avg over 2/6 total attempts. Reasons: P1 Attem...


```markdown
# @title 8. 生成された回答ファイルと評価結果ファイルのダウンロード

生成された回答ファイル (`raw_answers_base_llm.md`, `raw_answers_rag.md`) および評価結果ファイル (`evaluation_results.csv`) は、Colabのファイルシステム上にあります。
これらのファイルをダウンロードして、ローカルで詳細な分析や考察を進めることができます。

ダウンロード方法：

1. Colabの左側にあるファイルアイコンをクリックしてファイルブラウザを開きます。
2. リポジトリのディレクトリ (`YourRepositoryName`) を展開します。
3. `results` フォルダを開きます。
4. ダウンロードしたいファイル (例: `evaluation_results.csv`) を右クリックし、「ダウンロード」を選択します。
```

---